# Исследование надёжности заёмщиков

Заказчик — кредитный отдел банка. Нужно разобраться, влияет ли семейное положение и количество детей клиента на факт погашения кредита в срок. Входные данные от банка — статистика о платёжеспособности клиентов.

Результаты исследования будут учтены при построении модели **кредитного скоринга** — специальной системы, которая оценивает способность потенциального заёмщика вернуть кредит банку.

## Загрузка и просмотр данных
Для начала, импортируем библиотеку pandas, необходимую для обработки и анализа данных:

In [1]:
import pandas as pd

In [49]:
import warnings

warnings.filterwarnings('ignore')
# позволяет игнорировать предупреждения от указанного модуля.

Прочитаем файл с данными data.csv и сохраним его в переменной credits_data  
просмотрим первые ее 10 строк:

In [2]:
credits_data = pd.read_csv('/datasets/data.csv')

credits_data.head(10)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу
5,0,-926.185831,27,высшее,0,гражданский брак,1,M,компаньон,0,255763.565419,покупка жилья
6,0,-2879.202052,43,высшее,0,женат / замужем,0,F,компаньон,0,240525.971920,операции с жильем
7,0,-152.779569,50,СРЕДНЕЕ,1,женат / замужем,0,M,сотрудник,0,135823.934197,образование
8,2,-6929.865299,35,ВЫСШЕЕ,0,гражданский брак,1,F,сотрудник,0,95856.832424,на проведение свадьбы
9,0,-2188.756445,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425.938277,покупка жилья для семьи


Получим общую информацию о данных из таблицы credits_data

In [3]:
credits_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       19351 non-null float64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        19351 non-null float64
purpose             21525 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


<ins>Рассмотрим подробнее полученную информацию.</ins>

Всего в таблице 12 столбцов.  
Типы данных столбцов: < int64 >, < float64 > и < object >.  
Количество значений в столбцах различается. Это говорит о том, что в данных есть пропущенные значения.  

Подробно разберем, какие в credits_data столбцы, и какую информацию они содержат:

* children — количество детей в семье
* days_employed — общий трудовой стаж в днях
* dob_years — возраст клиента в годах
* education — уровень образования клиента
* education_id — идентификатор уровня образования
* family_status — семейное положение
* family_status_id — идентификатор семейного положения
* gender — пол клиента
* income_type — тип занятости
* debt — имел ли задолженность по возврату кредитов
* total_income — ежемесячный доход
* purpose — цель получения кредита

Судя по первым 10 строкам таблицы, визуально можно выделить следующие ошибки в столбцах:  
1. разный регистр букв в одинаковых словах в столбце education (уровень образования клиента);    
2. отрицательные значения в столбце days_employed (трудовой стаж клиента)
3. различные формулировки одинаковых по

**Вывод**

Таблица credits_data хранит информацию, отображающую статистику кредитоспособности клиентов банка-заказчика.   

Каждая строка таблицы содержит информацию об одном клиенте: пол, возраст, образование, тип занятости и трудовой стаж, семейное положение и количество детей, а также его ежемесячный доход и данные о кредитах, таких как: цель получения и наличие задолженностей по возврату.  

Мы выделили две проблемы, которые нужно решить: разный регистр букв в одинаковых словах и отрицательные значения, противоречащие здравому смыслу. Также необходимо будет проверить таблицу на пропуски и дубликаты.

Т.к. перед нами стоит задача: разобраться, влияет ли семейное положение и количество детей клиента на факт погашения кредита в срок, для проверки рабочих гипотез, особенно ценны будут столбцы family_status, children и debt.

## Предобработка данных

### Обработка пропусков

Проверим перечень названия столбцов на корректность и отсутствие проблем:

In [4]:
credits_data.columns

Index(['children', 'days_employed', 'dob_years', 'education', 'education_id',
       'family_status', 'family_status_id', 'gender', 'income_type', 'debt',
       'total_income', 'purpose'],
      dtype='object')

Наименования столбцов введены корректно, следовательно доступ к данным по названию столбца не будет осложнён.

Проверим данные на наличие пропусков.

In [5]:
credits_data.isna().sum()

children               0
days_employed       2174
dob_years              0
education              0
education_id           0
family_status          0
family_status_id       0
gender                 0
income_type            0
debt                   0
total_income        2174
purpose                0
dtype: int64

В столбцах days_employed и total_income обнаружены пустые значения, которые свидетельствуют о том, что для некоторых клиентов банка недоступна информация об их трудовом стаже и ежемесячном доходе.  

Причины пропусков одновременно в двух этих столбцах могут быть в том, что человек никогда официально не работал и, в результате, не обладает трудовым стажем, или же произошли какие-то проблемы с записью данных, что является наиболее вероятной причиной.  

Перед тем, как производить замену пропусков, необходимо сделать положительными все отрицательные значения в столбце days_employed с помощью модуля:

In [6]:
credits_data['days_employed'] = credits_data['days_employed'].abs()
credits_data.head()

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу


Разберемся со слишком большими значениями в столбце days_employed.  
Для начала сгенерируем по ним новый столбец years_employed, отображающий стаж в годах:

In [7]:
credits_data['years_employed'] = credits_data['days_employed'] / 365
credits_data.sort_values(by='years_employed', ascending=False).head()

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,years_employed
6954,0,401755.400475,56,среднее,1,вдовец / вдова,2,F,пенсионер,0,176278.441171,ремонт жилью,1100.699727
10006,0,401715.811749,69,высшее,0,Не женат / не замужем,4,F,пенсионер,0,57390.256908,получение образования,1100.591265
7664,1,401675.093434,61,среднее,1,женат / замужем,0,F,пенсионер,0,126214.519212,операции с жильем,1100.479708
2156,0,401674.466633,60,среднее,1,женат / замужем,0,M,пенсионер,0,325395.724541,автомобили,1100.477991
7794,0,401663.850046,61,среднее,1,гражданский брак,1,F,пенсионер,0,48286.441362,свадьба,1100.448904


In [8]:
credits_data[(credits_data['dob_years'] - credits_data['years_employed']) < 14]['years_employed'].count()

3535

In [9]:
credits_data['dob_years'].max()

75

In [10]:
count_hardworkers = credits_data[credits_data['years_employed'] > 75]['years_employed'].count()
count_hardworkers

3445

In [11]:
percent = count_hardworkers/len(credits_data.index)

print('Таким образом, судя по данным из датасета, у {:.0%} клиентов трудовой стаж превышает все возможные границы.'.format(percent))

Таким образом, судя по данным из датасета, у 16% клиентов трудовой стаж превышает все возможные границы.


Мы исследовали данные и отметили, что:
- указанный стаж не соответствует законодательству РФ (минимальный порог вхождения - с 14 лет)
- указанный стаж превышает возраст клиента и вообще может быть указан как 75 и более лет (а максимальный 1100 лет), при этом максимальный возраст клиента равен 75 годам

16% - это значимый процент выборки и просто так удалить строки мы не можем. Но т.к. при проверке основной гипотезы данный столбец не представляет важности, можно заменить их на среднее (mean)

In [12]:
mean_days = int(credits_data[credits_data['years_employed'] <= 75]['days_employed'].mean())
credits_data.loc[credits_data.years_employed > 75, 'days_employed'] = mean_days

In [13]:
credits_data.drop('years_employed', axis=1, inplace=True) 

In [14]:
credits_data.head()

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,2353.000000,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу


Проверим столбец dob_years (возраст клиента) на нулевые значения:

In [15]:
credits_data.loc[credits_data.dob_years == 0, 'dob_years'].count()

101

Полученное количество строк с нулями занимает меньше чем 1% от всей выборки и удаление строк с ними не повлияет на результат нашего исследования. Поэтому удалим их и проверим что нулей в столбце больше не осталось:

In [16]:
credits_data = credits_data[credits_data.dob_years > 0]

credits_data.loc[credits_data.dob_years == 0, 'dob_years'].count()

0

In [17]:
credits_data['children'].value_counts()

 0     14080
 1      4802
 2      2042
 3       328
 20       75
-1        47
 4        41
 5         9
Name: children, dtype: int64

Данный столбец важен нам для дальнейшего анализа, поэтому необходимо удалить единственный артефакт: количество детей равное 20 (таких строк 75 и это незначительное количество для нашей выборки), а строки с отрицательным показателем детей взять по модулю.

In [18]:
# возьмем модуль от значений столбца и перезапишем его присваиванием:
credits_data['children'] = credits_data['children'].abs()

# исключим строки с 20ю детьми из датафрема:
credits_data = credits_data[credits_data.children < 20]

credits_data['children'].value_counts()

0    14080
1     4849
2     2042
3      328
4       41
5        9
Name: children, dtype: int64

Произведем замену пропусков в столбцах:  
- в столбце days_employed - на средние значения;  
- в столбце total_income - на медианные значения.

In [19]:
credits_data.isna().sum()

children               0
days_employed       2155
dob_years              0
education              0
education_id           0
family_status          0
family_status_id       0
gender                 0
income_type            0
debt                   0
total_income        2155
purpose                0
dtype: int64

In [20]:
mean_days_employed = credits_data.sort_values(by='days_employed')['days_employed'].mean()
credits_data['days_employed'] = credits_data['days_employed'].fillna(mean_days_employed)

In [21]:
median_total_income = credits_data.sort_values(by='total_income')['total_income'].median()
credits_data['total_income'] = credits_data['total_income'].fillna(median_total_income)

Убедимся, что пропущенных значений больше не осталось:

In [22]:
credits_data.isna().sum()

children            0
days_employed       0
dob_years           0
education           0
education_id        0
family_status       0
family_status_id    0
gender              0
income_type         0
debt                0
total_income        0
purpose             0
dtype: int64

Перепишем индексы в датафрейме:

In [23]:
credits_data.reset_index(drop = True)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,2353.000000,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу
...,...,...,...,...,...,...,...,...,...,...,...,...
21344,1,4529.316663,43,среднее,1,гражданский брак,1,F,компаньон,0,224791.862382,операции с жильем
21345,0,2353.000000,67,среднее,1,женат / замужем,0,F,пенсионер,0,155999.806512,сделка с автомобилем
21346,1,2113.346888,38,среднее,1,гражданский брак,1,M,сотрудник,1,89672.561153,недвижимость
21347,3,3112.481705,38,среднее,1,женат / замужем,0,M,сотрудник,1,244093.050500,на покупку своего автомобиля


**Вывод**

На данном этапе, мы обработали некорректные значения (отрицательный стаж, слишком большой стаж, нулевой возраст) и заменили пропущенные значения, что, в дальнейшем, позволит нам провести анализ более точно. 

### Замена типа данных

Просмотрим все имеющиеся типы данных столбцов датафрейма:

In [24]:
credits_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 21349 entries, 0 to 21524
Data columns (total 12 columns):
children            21349 non-null int64
days_employed       21349 non-null float64
dob_years           21349 non-null int64
education           21349 non-null object
education_id        21349 non-null int64
family_status       21349 non-null object
family_status_id    21349 non-null int64
gender              21349 non-null object
income_type         21349 non-null object
debt                21349 non-null int64
total_income        21349 non-null float64
purpose             21349 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 2.1+ MB


Произведем замену вещественного типа данных столбцов days_employed и total_income на целочисленный:

In [25]:
credits_data['days_employed'] = credits_data['days_employed'].astype('int')
credits_data['total_income'] = credits_data['total_income'].astype('int')

credits_data['days_employed'].dtype

dtype('int64')

In [26]:
credits_data['total_income'].dtype

dtype('int64')

**Вывод**

Для замены вещественного типа данных на целочисленный, нами был выбран метод .astype() с присваиванием, т.к. он наиболее удобен и выполняет поставленную задачу при помощи передачи типа в качестве аргумента.

### Обработка дубликатов

In [27]:
credits_data.head(10)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,8437,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875,покупка жилья
1,1,4024,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,приобретение автомобиля
2,0,5623,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885,покупка жилья
3,3,4124,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628,дополнительное образование
4,0,2353,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616,сыграть свадьбу
5,0,926,27,высшее,0,гражданский брак,1,M,компаньон,0,255763,покупка жилья
6,0,2879,43,высшее,0,женат / замужем,0,F,компаньон,0,240525,операции с жильем
7,0,152,50,СРЕДНЕЕ,1,женат / замужем,0,M,сотрудник,0,135823,образование
8,2,6929,35,ВЫСШЕЕ,0,гражданский брак,1,F,сотрудник,0,95856,на проведение свадьбы
9,0,2188,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425,покупка жилья для семьи


Просмотрев данные еще раз, мы видим в столбце education записи с разным регистром.  
Проверим сколько таких записей найдется при помощи метода .value_counts(), который подсчитает частоту встречаемости всех уникальных объектов столбца.

In [28]:
credits_data['education'].value_counts()

среднее                13640
высшее                  4674
СРЕДНЕЕ                  768
Среднее                  702
неоконченное высшее      664
ВЫСШЕЕ                   271
Высшее                   266
начальное                250
Неоконченное высшее       47
НЕОКОНЧЕННОЕ ВЫСШЕЕ       29
НАЧАЛЬНОЕ                 17
Начальное                 15
ученая степень             4
УЧЕНАЯ СТЕПЕНЬ             1
Ученая степень             1
Name: education, dtype: int64

Чтобы учесть такие дубликаты, приведем все символы строк данного столбца к нижнему регистру с помощью метода .str.lower() из Pandas:

In [29]:
credits_data['education'] = credits_data['education'].str.lower()

credits_data['education'].value_counts()

среднее                15110
высшее                  5211
неоконченное высшее      740
начальное                282
ученая степень             6
Name: education, dtype: int64

Мы повторно просмотрели все уникальные значения столбца education и убедились в корректном удалении дубликатов.

**Вывод**

В данном пукте мы обработали дубликаты, найденные в столбце education с помощью приведения к одному регистру. Для этого нами был использован метод .str.lower(), т.к. он обрабатывает данные в Series и нам подходит.

Строки датасета, характеризующие клиентов банка, не имеют уникальных идентификаторов (к примеру, паспортные данные) и, по этой причине, не позволяют выявить дублированные строки, если даже они есть.  
С учетом описания данных в столбцах, одинаковые по содержимому строки могут представлять характеристики двух совершенно разных клиентов и, поэтому, даже при их наличии, их не следует удалять, т.к. это действие неблагоприятно повлияет на дальнейший анализ данных.

### Лемматизация

Для столбца произведем лемматизацию, т.е. приведение слова к его словарной форме (лемме).  
Для выполнения этой задачи, импортируем библиотеку с лемматизацией на русском языке pymystem3:

In [30]:
from pymystem3 import Mystem
m = Mystem()

lemmas = []
purposes = credits_data['purpose'].unique()

def to_lemmas(purposes):
    lemmas = ' '.join(m.lemmatize(purposes)).rstrip()
    return lemmas
  
credits_data['lemmas'] = credits_data['purpose'].apply(to_lemmas)

credits_data.head(10)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,lemmas
0,1,8437,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875,покупка жилья,покупка жилье
1,1,4024,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,приобретение автомобиля,приобретение автомобиль
2,0,5623,33,среднее,1,женат / замужем,0,M,сотрудник,0,145885,покупка жилья,покупка жилье
3,3,4124,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628,дополнительное образование,дополнительный образование
4,0,2353,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616,сыграть свадьбу,сыграть свадьба
5,0,926,27,высшее,0,гражданский брак,1,M,компаньон,0,255763,покупка жилья,покупка жилье
6,0,2879,43,высшее,0,женат / замужем,0,F,компаньон,0,240525,операции с жильем,операция с жилье
7,0,152,50,среднее,1,женат / замужем,0,M,сотрудник,0,135823,образование,образование
8,2,6929,35,высшее,0,гражданский брак,1,F,сотрудник,0,95856,на проведение свадьбы,на проведение свадьба
9,0,2188,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425,покупка жилья для семьи,покупка жилье для семья


Для проведения лематизации по столбцу целей кредита (purpose), мы создали список уникальных значений по столбцу purpose и с помощью метода apply(), в который передали функцию с лемматизацией целей кредита, прошлись по всем строкам датасета, сгенерировав на основании столбца purpose новый столбец содержащий леммы целей.  
Метод .join() использовали для представления содержимого ячеек столбца в приятном для восприятия виде.  
Подсчитаем уникалльные леммы методом Counter:

In [31]:
lemmas_list = (' '.join(credits_data['lemmas'].unique())).split()

from collections import Counter
Counter(lemmas_list)

Counter({'покупка': 10,
         'жилье': 7,
         'приобретение': 1,
         'автомобиль': 8,
         'дополнительный': 2,
         'образование': 9,
         'сыграть': 1,
         'свадьба': 3,
         'операция': 4,
         'с': 5,
         'на': 4,
         'проведение': 1,
         'для': 2,
         'семья': 1,
         'недвижимость': 10,
         'коммерческий': 2,
         'жилой': 2,
         'строительство': 3,
         'собственный': 1,
         'подержать': 1,
         'свой': 4,
         'со': 1,
         'заниматься': 2,
         'сделка': 2,
         'подержанный': 1,
         'получение': 3,
         'высокий': 3,
         'профильный': 1,
         'сдача': 1,
         'ремонт': 1})

In [32]:
credits_data['lemmas'].value_counts()

автомобиль                                    965
свадьба                                       791
на   проведение   свадьба                     769
сыграть   свадьба                             765
операция   с   недвижимость                   672
покупка   коммерческий   недвижимость         658
покупка   жилье   для   сдача                 650
операция   с   коммерческий   недвижимость    645
операция   с   жилье                          643
покупка   жилье   для   семья                 639
покупка   жилье                               638
жилье                                         637
операция   со   свой   недвижимость           629
недвижимость                                  629
строительство   собственный   недвижимость    627
строительство   жилой   недвижимость          622
строительство   недвижимость                  620
покупка   свой   жилье                        619
покупка   недвижимость                        619
ремонт   жилье                                608


Судя по получившимся леммам и их анализу, можно кратко сформулировать 5 целей получения кредита:

- свадьба  
- недвижимость и жилье  
- образование  
- автомобиль
- остальное

**Вывод**

В данном пункте мы провели лемматизацию целей взятия кредита клиентами банка и благодаря этому сформулировали кратко из большого множества всего 5 целей.

### Категоризация данных

В данном пункте нам предстоит выяснить и сделать выводы о том, связаны ли такие характеристики клиента как семейное положение и количество детей на факт погашения кредита.

Нам пригодится информация из следующих столбцов датафрейма:

- children — количество детей в семье
- family_status — семейное положение
- family_status_id — идентификатор семейного положения
- debt —  наличие задолженности по кредиту



- **Детность**

Проведем категоризацию по столбцу children.  
Для начала посмотрим на уникальные значения столбца и их количество при помощи метода .value_counts():

In [33]:
credits_data['children'].value_counts()

0    14080
1     4849
2     2042
3      328
4       41
5        9
Name: children, dtype: int64

Создадим функцию для последующего четкого разделения клиентов на категории в соответствии с количеством их детей.  
Применим функцию для прохожждения по столбцу children и записи детности клиентов в отдельный столбец childishness методом .apply

In [34]:
def childishness(children):
    """
    Возвращает детность семьи клиента по значению children, используя правила:
    - 'бездетный' - при значении children равном нулю
    - 'многодетный' - при значении children больше двух
    - 'малодетный' - в остальных случаях

    """
    
    if children == 0:
        return 'бездетный'
    if children > 2:
        return 'многодетный'
    return 'малодетный'

credits_data['childishness'] = credits_data['children'].apply(childishness)
credits_data.head()

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,lemmas,childishness
0,1,8437,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875,покупка жилья,покупка жилье,малодетный
1,1,4024,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,приобретение автомобиля,приобретение автомобиль,малодетный
2,0,5623,33,среднее,1,женат / замужем,0,M,сотрудник,0,145885,покупка жилья,покупка жилье,бездетный
3,3,4124,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628,дополнительное образование,дополнительный образование,многодетный
4,0,2353,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616,сыграть свадьбу,сыграть свадьба,бездетный


- **Семейное положение**

Далее, по тому же принципу что и с детностью, посмотрим на уникальные значения столбца family_status (семейное положение) и их количество при помощи метода .value_counts():

In [35]:
credits_data['family_status'].value_counts()

женат / замужем          12283
гражданский брак          4144
Не женат / не замужем     2788
в разводе                 1183
вдовец / вдова             951
Name: family_status, dtype: int64

In [36]:
family_status_data = credits_data.groupby(['family_status', 'debt']).agg({'family_status': ['count']})
family_status_data

family_status
                                   count
family_status         debt              
Не женат / не замужем 0             2516
                      1              272
в разводе             0             1099
                      1               84
вдовец / вдова        0              889
                      1               62
гражданский брак      0             3761
                      1              383
женат / замужем       0            11359
                      1              924

В столбце family_status слишком много вариаций семейного положения, что затрудняет дальнейший анализ.  

Объединим близкие по смыслу значения при помощи функции family_status_func, которая примет в качестве аргумента значения строк из столбца family_status_id (т.к. нам удобнее работать с числами из family_status_id, чем со строками из столбца family_status).

Для начала, для корректного описания функции, определим каким family_status_id соответствуют какие family_status при помощи группировки по двум этим столбцам и заодно подсчитаем количество по каждому семейному положению. 

In [37]:
credits_data.groupby(['family_status', 'family_status_id'])['family_status'].count()

family_status          family_status_id
Не женат / не замужем  4                    2788
в разводе              3                    1183
вдовец / вдова         2                     951
гражданский брак       1                    4144
женат / замужем        0                   12283
Name: family_status, dtype: int64

Опишем функцию family_status_func и запишем результат ее выполнения в столбец family_status_common.  
Выведем первые 10 строк при помощи метода .head(), чтобы убедиться что все сработало:

In [38]:
def family_status_func(family_status_id):
    """
    Возвращает более точное семейное положение клиента по значению family_status_id, используя правила:
    - 'холост' - при значении family_status_id больше 1
    - 'в браке' - в остальных случаях

    """
    
    if family_status_id > 1:
        return 'холост'
    return 'в браке'

credits_data['family_status_common'] = credits_data['family_status_id'].apply(family_status_func)
credits_data.head()

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,lemmas,childishness,family_status_common
0,1,8437,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875,покупка жилья,покупка жилье,малодетный,в браке
1,1,4024,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,приобретение автомобиля,приобретение автомобиль,малодетный,в браке
2,0,5623,33,среднее,1,женат / замужем,0,M,сотрудник,0,145885,покупка жилья,покупка жилье,бездетный,в браке
3,3,4124,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628,дополнительное образование,дополнительный образование,многодетный,в браке
4,0,2353,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616,сыграть свадьбу,сыграть свадьба,бездетный,в браке


- **Доход**

Посмотрим на максимальное, минимальное и медианное значения столбца total_income (доход в месяц) при помощи метода .value_counts():

In [39]:
credits_data['total_income'].max()

2265604

In [40]:
credits_data['total_income'].min()

20667

In [41]:
credits_data.sort_values(by='total_income')['total_income'].median()

145020.0

In [42]:
def finance_def(total_income):
    """
    Возвращает название группы по уровню дохода клиента:
    - 'низкий доход' - при total_income < 50 000
    - 'высокий доход' - при total_income > 150 000
    - 'средний доход' - в остальных случаях
    """
    
    if total_income < 50000:
        return 'низкий доход'
    if total_income > 150000:
        return 'высокий доход'
    return 'средний доход'

credits_data['total_income_group'] = credits_data['total_income'].apply(finance_def)
credits_data.head()

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,lemmas,childishness,family_status_common,total_income_group
0,1,8437,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875,покупка жилья,покупка жилье,малодетный,в браке,высокий доход
1,1,4024,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,приобретение автомобиля,приобретение автомобиль,малодетный,в браке,средний доход
2,0,5623,33,среднее,1,женат / замужем,0,M,сотрудник,0,145885,покупка жилья,покупка жилье,бездетный,в браке,средний доход
3,3,4124,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628,дополнительное образование,дополнительный образование,многодетный,в браке,высокий доход
4,0,2353,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616,сыграть свадьбу,сыграть свадьба,бездетный,в браке,высокий доход


- **Цели кредита**

Посмотрим на уникальные значения столбца lemmas (леммы целей кредита) и их количество при помощи метода .value_counts():

In [43]:
credits_data['lemmas'].value_counts()

автомобиль                                    965
свадьба                                       791
на   проведение   свадьба                     769
сыграть   свадьба                             765
операция   с   недвижимость                   672
покупка   коммерческий   недвижимость         658
покупка   жилье   для   сдача                 650
операция   с   коммерческий   недвижимость    645
операция   с   жилье                          643
покупка   жилье   для   семья                 639
покупка   жилье                               638
жилье                                         637
операция   со   свой   недвижимость           629
недвижимость                                  629
строительство   собственный   недвижимость    627
строительство   жилой   недвижимость          622
строительство   недвижимость                  620
покупка   свой   жилье                        619
покупка   недвижимость                        619
ремонт   жилье                                608


Дополним датафрейм столбцом, сформированным по основным выделенным нами ранее леммам (образование, автомобиль, свадьба, жилье/недвижимость). 

Для этого напишем функцию, которая будет принимать в качестве аргумента строку из столбца lemmas и возвращать значение, в соответствии с выполнившимся условием if.

Пройдемся по строкам столбца 'lemmas', содержащего леммы целей получения дохода, и сформируем по его данным с помощью функции cut_purpose() новый столбец 'cut_purposes'.

Выведем первые строки таблицы, чтобы убедиться в том что все прошло успешно.

In [44]:
def cut_purpose(lemmas):
    """
    Возвращает название группы по уровню дохода клиента:
    - 'образование' - при вхождении леммы 'образование' в lemmas
    - 'автомобиль' - при вхождении леммы 'автомобиль' в lemmas
    - 'свадьба' - при вхождении леммы 'свадьба' в lemmas
    - 'недвижимость' - в остальных случаях
    """
    
    if 'образование' in lemmas:
        return 'образование'
    if 'автомобиль' in lemmas:
        return 'автомобиль'
    if 'свадьба' in lemmas:
        return 'свадьба'
    return 'недвижимость'

credits_data['cut_purposes'] = credits_data['lemmas'].apply(cut_purpose)
credits_data.head()

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,lemmas,childishness,family_status_common,total_income_group,cut_purposes
0,1,8437,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875,покупка жилья,покупка жилье,малодетный,в браке,высокий доход,недвижимость
1,1,4024,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,приобретение автомобиля,приобретение автомобиль,малодетный,в браке,средний доход,автомобиль
2,0,5623,33,среднее,1,женат / замужем,0,M,сотрудник,0,145885,покупка жилья,покупка жилье,бездетный,в браке,средний доход,недвижимость
3,3,4124,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628,дополнительное образование,дополнительный образование,многодетный,в браке,высокий доход,образование
4,0,2353,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616,сыграть свадьбу,сыграть свадьба,бездетный,в браке,высокий доход,свадьба


**Вывод**

В данном пункте мы провели категоризацию - объединение данных в категории - по столбцам с семейным положением, доходами клиентов, количеству детей при помощи функций и метода .apply(). В результате чего данные представлены в удобном для анализа виде.

## Ответы на поставленные вопросы

- **Есть ли зависимость между наличием детей и возвратом кредита в срок?**

Просмотрим таблицу соответствия количества клиентов с их детностью:

In [45]:
credits_data['childishness'].value_counts()

бездетный      14080
малодетный      6891
многодетный      378
Name: childishness, dtype: int64

Посмотрим по столбцу debt сколько клиентов отдали кредит во время (0), а сколько человек имели задолженность по кредиту (1):

In [46]:
credits_data['debt'].value_counts()

0    19624
1     1725
Name: debt, dtype: int64

Сагрегируем данные по столбцу детность (childishness) и посмотрим должников по кредитам:

In [47]:
childishness_data = credits_data.groupby(['childishness', 'debt']).agg({'childishness': ['count']})
childishness_data

childishness
                         count
childishness debt             
бездетный    0           13022
             1            1058
малодетный   0            6255
             1             636
многодетный  0             347
             1              31

In [48]:
credits_data['childishness'].value_counts()

бездетный      14080
малодетный      6891
многодетный      378
Name: childishness, dtype: int64

Посчитаем процентное соотношение отдавших кредит во время и должников к общему числу клиентов в каждой исследуемой группе. Далее сравним процент должников с процентом добросовестных клиентов, чтобы сформулировать выводы.

Пройдем циклом по значениям таблицы соответствия количества клиентов с их детностью, используя условия отбора (детность из первого столбца Series и наличие долга в соответствующей группе). Далее высчитаем процент и посмотрим на полученный результат:

In [50]:
for index, value in credits_data['childishness'].value_counts().items():
    # высчитываем количество должников по группе детности:
    debtor = credits_data[credits_data.childishness == index][credits_data.debt == 1]['debt'].count()
    # получаем отношение должников данной группы детности к общему объему клиентов из этой группы:
    percent = debtor / value;
    # выводим процент должников от общего количества клиентов группы детности:
    print('{0}:\t{1:.2%}'.format(index, percent))

бездетный:	7.51%
малодетный:	9.23%
многодетный:	8.20%


**Вывод**

По полученным данным соотношение клиентов с задолженностью и без нее для всех категорий детности приблизительно одинакова, но можно предположить, что количество детей незначительным образом, но влияет на возврат кредита в срок: люди имеющие детей менее дисциплинированы и склонны отдавать кредит в срок реже, чем бездетные клиенты банка.

- **Есть ли зависимость между семейным положением и возвратом кредита в срок?**

Сделаем агрегацию данных, сгруппированных по столбцам 'family_status_common' и 'debt'.  
Проанализируем то что получили:

In [51]:
family_status_data1 = credits_data.groupby(['family_status_common', 'debt']).agg({'family_status_common': ['count']})
family_status_data1

family_status_common
                                         count
family_status_common debt                     
в браке              0                   15120
                     1                    1307
холост               0                    4504
                     1                     418

In [52]:
credits_data['family_status_common'].value_counts()

в браке    16427
холост      4922
Name: family_status_common, dtype: int64

По тому же принципу что и с детностью, посчитаем процентное соотношение отдавших кредит во время и должников к общему числу клиентов в каждой исследуемой группе. Далее сравним процент должников с процентом добросовестных клиентов, чтобы сформулировать выводы:

In [53]:
for index, value in credits_data['family_status_common'].value_counts().items():
    # высчитываем количество должников по группе:
    debtor = credits_data[credits_data.family_status_common == index][credits_data.debt == 1]['debt'].count()
    # получаем отношение должников данной группы к общему объему клиентов из этой группы:
    percent = debtor / value;
    # выводим процент должников от общего количества клиентов группы:
    print('{0}: {1:.2%}'.format(index, percent))

в браке: 7.96%
холост: 8.49%


**Вывод**

По полученным данным соотношение должников к добросовестным клиентам в обеих группах примерно равно, но стоит сделать предположение, что люди в браке склонны отдавать кредит во время, когда как не женатые/не замужние клиенты более часто задерживают возврат кредита в срок.

- **Есть ли зависимость между уровнем дохода и возвратом кредита в срок?**

Сделаем агрегацию данных, сгруппированных по столбцам 'family_status_common' и 'debt'.
Проанализируем то что получили:

In [54]:
total_income_data = credits_data.groupby(['total_income_group', 'debt']).agg({'total_income': ['count']})
total_income_data

total_income
                               count
total_income_group debt             
высокий доход      0            8391
                   1             718
низкий доход       0             347
                   1              23
средний доход      0           10886
                   1             984

In [55]:
credits_data['total_income_group'].value_counts()

средний доход    11870
высокий доход     9109
низкий доход       370
Name: total_income_group, dtype: int64

Посчитаем процентное соотношение отдавших кредит во время и должников к общему числу клиентов в каждой исследуемой группе. Далее сравним процент должников с процентом добросовестных клиентов, чтобы сформулировать выводы:

In [56]:
for index, value in credits_data['total_income_group'].value_counts().items():
    # высчитываем количество должников по группе:
    debtor = credits_data[credits_data.total_income_group == index][credits_data.debt == 1]['debt'].count()
    # получаем отношение должников данной группы к общему объему клиентов из этой группы:
    percent = debtor / value;
    # выводим процент должников от общего количества клиентов группы:
    print('{0}: {1:.2%}'.format(index, percent))

средний доход: 8.29%
высокий доход: 7.88%
низкий доход: 6.22%


**Вывод**

Исследуя получившуюся информацию, можно подтвердить предположение о наличии зависимости между доходом клиента и возвратом кредита в срок: люди со средним и высоким доходом чаще задерживают кредит, чем люди с низким доходом.

- **Как разные цели кредита влияют на его возврат в срок?**

Сделаем агрегацию данных, сгруппированных по столбцам 'cut_purposes' и 'debt'.
Проанализируем то что получили:

In [57]:
cut_purposes_data = credits_data.groupby(['cut_purposes', 'debt']).agg({'cut_purposes': ['count']})
cut_purposes_data

cut_purposes
                         count
cut_purposes debt             
автомобиль   0            3880
             1             398
недвижимость 0            9980
             1             777
образование  0            3620
             1             369
свадьба      0            2144
             1             181

Посмотрим общее количество клиентов по каждой группе:

In [58]:
credits_data['cut_purposes'].value_counts()

недвижимость    10757
автомобиль       4278
образование      3989
свадьба          2325
Name: cut_purposes, dtype: int64

Посчитаем процентное соотношение отдавших кредит во время и должников к общему числу клиентов в каждой исследуемой группе. Далее сравним процент должников с процентом добросовестных клиентов, чтобы сформулировать выводы:

In [59]:
for index, value in credits_data['cut_purposes'].value_counts().items():
    # высчитываем количество должников по группе:
    debtor = credits_data[credits_data.cut_purposes == index][credits_data.debt == 1]['debt'].count()
    # получаем отношение должников данной группы к общему объему клиентов из этой группы:
    percent = debtor / value;
    # выводим процент должников от общего количества клиентов группы:
    print('{0}: {1:.2%}'.format(index, percent))

недвижимость: 7.22%
автомобиль: 9.30%
образование: 9.25%
свадьба: 7.78%


**Вывод**

Исследуя получившуюся информацию, можно сделать вывод о том, что клиенты, взявшие кредит на автомобиль или образорвание чаще склонны к задержке возврата кредита в срок, чем клиенты взявшие кредит на свадьбу или недвижимость.

## Общий вывод

Проделав большую работу над данным проектом, мы с уверенностью можем сказать о важности каждого этапа исследования на результат и формулирование четких выводов.  

С помощью всех этапов предобработки данных, мы провели исследование информации о кредитной истории клиентов, которую предоставил кредитный отдел банка, и теперь можем сформулировать вывод о том что **семейное положение и количество детей клиента все-таки влияет на факт возврата кредита в срок**.  

- Не женатые / не замужние люди склонны чаще задерживать выплату кредита в срок.
- Чем больше детей у клиента, тем в большей вероятности это способствует задержке возврата кредита.

Также на возрат кредита в срок могут повлиять цели кредита и уровень дохода клиента:

- Клиенты, берущие кредит на автомобиль или образование чаще задерживают возврат кредита, чем клиенты с целями, связанными с недвижимостью и свадьбой.
- Клиенты с доходом более 50ти тысяч склонны чаще задерживать выплату кредита в срок, чем клиенты с низким уровнем дохода (ниже 50ти тысяч в месяц)

Таким образом по итогу мы можем сформировать  

**Портрет идеального заёмщика:**  
клиент состоит в официальном браке, без детей и имеющий низкий доход (менее 50тыс.руб.), осуществляющий заем денежных средств для покупки недвижимости или проведения свадьбы.

**Портрет клиента, не вернувшего долг по кредиту:**  
клиент не состоит в браке, имеет 1-2 детей, с доходом выше 50 тыс.руб в месяц (средний или высокий), осуществляющий заем средств на покупку автомобиля или оплату образования.
